In [1]:
import cfgrib
import numpy as np
import pandas as pd
import xarray as xr
import psyplot.project as psy
from scipy.interpolate import griddata
import scipy.interpolate as interp

import matplotlib.pyplot as plt
import iconarray
import psyplot.project as psy
import scipy.interpolate as interp  # type: ignore

from realtime_pollen_calibration import utils

In [ ]:
#####################################################
#
#        UTILITY FUNCTIONS
#
#####################################################

def read_atab(file_data, file_data_mod):   
    with open(file_data) as f:
        for n, line in enumerate(f):
            if line.strip()[0:8] == 'Latitude':
                lat_stns = np.fromstring(line.strip()[10:], sep=' ')
            if line.strip()[0:9] == 'Longitude':
                lon_stns = np.fromstring(line.strip()[11:], sep=' ')
            if line.strip()[0:18] == 'Missing_value_code':
                missing_value = float(line.strip()[20:])
            if line.strip()[0:9] == 'Indicator':
                stn_indicators = np.array(line.strip()[11:].split('\t'))
            if n == 16:
                break
    with open(file_data_mod) as f:
        for n, line in enumerate(f):
            if line.strip()[0:9] == 'Indicator':
                stn_indicators_mod = np.array(line.strip()[29:].split('         '))
                break
    [_, is1, is2] = np.intersect1d(stn_indicators, stn_indicators_mod, assume_unique=True, return_indices=True)
    istation_mod = is2[np.argsort(is1)]
    data = pd.read_csv(file_data, \
                       header = 17, \
                       delim_whitespace=True,\
                      parse_dates = [[1,2,3,4,5]])

    data_mod = pd.read_csv(file_data_mod, \
                       header = 18, \
                       delim_whitespace=True,\
                      parse_dates = [[3,4,5,6,7]])
    return data, data_mod, lat_stns, lon_stns, missing_value, istation_mod

def treat_missing(array, missing_value = -9999.0, tune_pol_default = 1., verbose = False):
    array_missing = array == missing_value
    skip_missing = np.count_nonzero(array_missing)
    Nstations = array.shape[1]
    if skip_missing > 0:
        for istation in range(Nstations):
            if np.count_nonzero(np.abs(array[:,istation] - missing_value) < 0.01)/len(array[:,istation]) < 0.1:
                idx1 = np.where(np.abs(array[:,istation] - missing_value) > 0.01)
                idx2 = np.where(np.abs(array[:,istation] - missing_value) < 0.01)
                if verbose:
                    print(f'Less than 10% of the data is missing, \
                    mean of the rest is: {np.mean(array[idx1, istation])}')
                array[idx2, istation] = np.mean(array[idx1, istation])
            else:
                if verbose:
                    print('More than 10% of the data is missing')
                array[:,istation] = tune_pol_default
    return array

def get_change_tune(
    pollen,
    array,
    array_mod,
    ds,
    lat_stns,
    lon_stns,
    istation_mod,
    tune_pol_default=1.0,
    eps=1e-2,
):
    nstns = array.shape[1]
    change_tune = np.ones(nstns)
    for istation in range(nstns):
        # sum of hourly observed concentrations of the last 5 days
        sum_obs = np.sum(array[:, istation])
        # sum of hourly modelled concentrations of the last 5 days
        sum_mod = np.sum(array_mod[:, istation_mod])
        # tuning factor at the current station
        tune_stns = get_field_at(
            ds,
            pollen + "tune",
            lat_stns[istation],
            lon_stns[istation],
            eps,
        )
        # saison days at the current station
        # if > 0 then the pollen season has started
        saisn_stns = get_field_at(
            ds,
            pollen + "saisn",
            lat_stns[istation],
            lon_stns[istation],
            eps,
        )
        if (saisn_stns > 0) and ((sum_obs <= 720) or (sum_mod <= 720)):
            change_tune[istation] = (tune_pol_default / tune_stns) ** (1 / 24)
        elif (saisn_stns > 0) and (sum_obs > 720) and (sum_mod > 720):
            change_tune[istation] = (sum_obs / sum_mod) ** (1 / 24)
    return change_tune

def interpolate(change, ds, field, lat_stns, lon_stns, method="multiply", ipollen=0):
    nstns = len(lat_stns)
    if method == "multiply":
        min_param = [3.389, 4.046, 7.738, 1.875]
        max_param = [0.235, 0.222, 0.216, 0.405]
    else:
        min_param = 1e10 * np.ones(4)
        max_param = -1e10 * np.ones(4)
    diff_lon = np.zeros((nstns,) + ds.longitude.shape)
    diff_lat = np.zeros((nstns,) + ds.longitude.shape)
    dist = np.zeros((nstns,) + ds.longitude.shape)
    for istation in range(nstns):
        diff_lon[istation, :] = (
            (ds.longitude - lon_stns[istation])
            * np.pi
            / 180
            * np.cos(ds.latitude * np.pi / 180)
        )
        diff_lat[istation, :] = (ds.latitude - lat_stns[istation]) * np.pi / 180
        dist[istation, :] = np.sqrt(
            diff_lon[istation, :] ** 2 + diff_lat[istation, :] ** 2
        )
    if method == "multiply":
        vec = np.maximum(
            np.minimum(
                ds[field].values
                * np.sum(change[:, np.newaxis, np.newaxis] / dist, axis=0)
                / np.sum(1 / dist, axis=0),
                min_param[ipollen],
            ),
            max_param[ipollen],
        )
    elif method == "sum":
        vec = np.maximum(
            np.minimum(
                ds[field].values
                + np.sum(change[:, np.newaxis, np.newaxis] / dist, axis=0)
                / np.sum(1 / dist, axis=0),
                min_param[ipollen],
            ),
            max_param[ipollen],
        )
    return vec

def plot_tunevec(ds, field, vec, lat_stns, lon_stns, filename = '', mode = 0):
    ds['vec'] = ds[field].copy(deep=True)
    ds.vec.values = vec
    
    # auxiliary plotting variables
    lonmin = np.amin(ds.longitude)
    lonmax = np.amax(ds.longitude)
    latmin = np.amin(ds.latitude)
    latmax = np.amax(ds.latitude)
    if mode == 1:
        MIN = 0.8#np.amin(vec)
        MAX = 1.2#np.amax(vec)
    elif mode == 0:
        MIN = np.amin(vec)
        MAX = np.amax(vec)
    elif mode ==2:
        MIN = -np.amax(np.abs(vec))
        MAX = np.amax(np.abs(vec))
    plot1 = ds.psy.plot.mapplot(
    name="vec",
    title="Alder Tuning Factor ",
    titlesize=15,
    map_extent = [lonmin, lonmax, latmin, latmax],
    bounds = {'method': 'minmax', 'N':100, 'vmin': MIN, 'vmax': MAX},
    cticks=np.linspace(MIN,MAX,6),
    lakes=True,
    borders=True,
    rivers=True,
    cticksize=8,
    clabel="Tuning Factor []",
    grid_labelsize=8,
    projection='robin',
    cmap='RdBu_r', xgrid = False, ygrid = False)
    fig = plt.gcf()
    Nstations = lon_stns.shape[0]
    pos_lon = np.zeros(Nstations)
    pos_lat = np.zeros(Nstations)
    vec_stns = np.zeros(Nstations)
    eps = 1e-2
    for istation in range(Nstations):
        pos_lon[istation],pos_lat[istation] = iconarray.add_coordinates(lon_stns[istation],lat_stns[istation],lonmin,lonmax,latmin,latmax)
        vec_stns[istation] = get_field_at(ds, 'vec', lat_stns[istation], lon_stns[istation])
    #fig.axes[0].scatter(pos_lon, pos_lat,s=20,color='w',marker='o', transform=fig.axes[0].transAxes) 
    fig.axes[0].scatter(pos_lon, pos_lat,s=50,color='k',marker='o', transform=fig.axes[0].transAxes) 
    fig.axes[0].scatter(pos_lon, pos_lat,s=20,c=vec_stns, marker='o', alpha=1, 
                        transform=fig.axes[0].transAxes, vmin=MIN, vmax = MAX, cmap = 'RdBu_r') 
    if len(filename) > 0:
        output_file = '/store/s83/gvanpary/Figures/' + filename
        plt.savefig(output_file)
    plot1.show()
    
def test_change_tune(pollen, change_tune, ds, ds2, lat_stns, lon_stns, eps = 1e-2):
    Nstations = len(lon_stns)
    tune_OLD = np.zeros(Nstations)
    tune_next = np.zeros(Nstations)
    for istation in range(Nstations):
        tune_OLD[istation] = get_field_at(ds, pollen + 'tune', lat_stns[istation], lon_stns[istation])
        tune_next[istation] = get_field_at(ds2, pollen + 'tune', lat_stns[istation], lon_stns[istation])
    change_tune_2 = tune_next/tune_OLD
    Err = change_tune_2 - change_tune
    return Err
'''
    Set change_tune to be coherent with the next timestep output
    in order to test the interpolation
    note that for this computation there are some approximations, i.e.
    AT one station 1/d_station >> 1/d_other_station and thus all the other
    terms in the interpolation are neglected.
'''
def test_interpolation(ds, ds2, lat_stns, lon_stns, eps = 1e-2):
    Nstations = len(lon_stns)
    tune_OLD = np.zeros(Nstations)
    tune_next = np.zeros(Nstations)
    for istation in range(Nstations):
        tune_OLD[istation] = ds.ALNUtune.where((np.abs(ds.longitude-lon_stns[istation]) < eps)\
                                & (np.abs(ds.latitude-lat_stns[istation]) < eps), drop=True).values[0][0]
        tune_next[istation] = ds2.ALNUtune.where((np.abs(ds2.longitude-lon_stns[istation]) < eps)\
                                & (np.abs(ds2.latitude-lat_stns[istation]) < eps), drop=True).values[0][0]
    change_tune_2 = tune_next/tune_OLD
    tune_vec_2 = interpolate(change_tune_2, ds, 'ALNUtune', lat_stns, lon_stns, 'mult')
    Err = ds2.ALNUtune - tune_vec_2
    return Err, change_tune_2, tune_vec_2

def get_field_at(ds, field, lat, lon, eps = 1e-2):
    dist = (ds.latitude - lat)**2 + (ds.longitude - lon)**2
    return ds[field].where(dist == dist.min(), drop = True)
#    return ds[field].where((np.abs(ds.longitude-lon) < eps)\
#                    & (np.abs(ds.latitude-lat) < eps), drop=True).values[0][0]

def interpolate2(change_tune, ds, lat_stns, lon_stns, epsilon = 100):
    nstns = len(lat_stns)
    stns_points = np.array([[lat_stns[i], lon_stns[i]] for i in range(nstns)])*np.pi/180
    grid_points = np.array(
        [ds.latitude.values.flatten(), ds.longitude.values.flatten()]
    ).T*np.pi/180
    change_tune_vec = interp.RBFInterpolator(
        stns_points, change_tune-1, kernel="gaussian", epsilon=epsilon, degree = -1, smoothing = 0,
    )(grid_points).reshape(ds.latitude.values.shape) + 1
    return change_tune_vec

In [ ]:
# debugging checks

# first check if get_change_tune is coherent with the next timestep output
istation = 2
print(f'Checking station n°{istation}')
eps = 1e-2
tune_OLD = ds.ALNUtune.where((np.abs(ds.longitude-lon_stns[istation]) < eps)\
                        & (np.abs(ds.latitude-lat_stns[istation]) < eps), drop=True).values[0][0]

print(f'Its old tune value was {tune_OLD}')
tune_NEW = ds.tune_vec.where((np.abs(ds.longitude-lon_stns[istation]) < eps)\
                        & (np.abs(ds.latitude-lat_stns[istation]) < eps), drop=True).values[0][0]

print(f'Its new tune value is {tune_NEW}')
print(f'The corresponding change_tune value is {change_tune[istation]}, \
(IF#1: {1/tune_OLD**(1/24)}, IF#2: {((np.sum(array[:,istation]) / np.sum(array_mod[:,istation_mod[istation]]))**(1/24))})')
print(f'Condition #1 would yield: {tune_OLD**(23/24)}')
print(f'Condition #2 would yield: {tune_OLD*((np.sum(array[:,istation]) / np.sum(array_mod[:,istation_mod[istation]]))**(1/24))}')
tune_next = ds2.ALNUtune.where((np.abs(ds.longitude-lon_stns[istation]) < eps)\
                        & (np.abs(ds.latitude-lat_stns[istation]) < eps), drop=True).values[0][0]
print(f'The new tune value should be according to the next timestep field {tune_next}')
print(f'And the corrresponding change_tune should be: {tune_next/tune_OLD}')
print('---------------------')


    


In [ ]:
# run tests

Err, change_tune_2, tune_vec_2 = test_interpolation(ds, ds2, lat_stns, lon_stns, eps = 1e-2)
#Err2 = test_change_tune(change_tune, ds, ds2, lat_stns, lon_stns, eps = 1e-2)

dtimesteps = ds.ALNUtune - ds2.ALNUtune

In [ ]:
Err = np.abs(Err)
ds['err'] = ds.ALNUtune.copy(deep=True)
ds.err.values = Err

# auxiliary plotting variables
lonmin = np.amin(ds.longitude)
lonmax = np.amax(ds.longitude)
latmin = np.amin(ds.latitude)
latmax = np.amax(ds.latitude)

MIN = np.amin(Err)
MAX = np.amax(Err)
plot1 = ds.psy.plot.mapplot(
name="err",
title="Residual error (ALNU tune field)",
map_extent = [lonmin, lonmax, latmin, latmax],
bounds = {'method': 'minmax', 'N':100, 'vmin': MIN, 'vmax': MAX},
cticks=np.linspace(MIN,MAX,6),
lakes=True,
borders=True,
rivers=True,
cticksize=8,
clabel="Tuning Factor []",
grid_labelsize=8,
projection='robin',
cmap='RdBu_r', xgrid = False, ygrid = False)
fig = plt.gcf()
Nstations = lon_stns.shape[0]
pos_lon = np.zeros(Nstations)
pos_lat = np.zeros(Nstations)
vec_stns = np.zeros(Nstations)
eps = 1e-2
for istation in range(Nstations):
    pos_lon[istation],pos_lat[istation] = iconarray.add_coordinates(lon_stns[istation],lat_stns[istation],lonmin,lonmax,latmin,latmax)
    vec_stns[istation] = ds.err.where((np.abs(ds.longitude-lon_stns[istation]) < eps)\
                    & (np.abs(ds.latitude-lat_stns[istation]) < eps), drop=True).values[0][0]
fig.axes[0].scatter(pos_lon, pos_lat,s=20,color='w',marker='o', transform=fig.axes[0].transAxes) 
fig.axes[0].scatter(pos_lon, pos_lat,s=10,color='k',marker='o', transform=fig.axes[0].transAxes) 
fig.axes[0].scatter(pos_lon, pos_lat,s=15,c=vec_stns,marker='o', alpha=0, transform=fig.axes[0].transAxes, vmin=MIN, vmax = MAX, cmap = 'RdBu_r') 

output_file = '/store/s83/gvanpary/Figures/' + 'interpolation_residual'
plt.savefig(output_file)
plot1.show()
    

In [ ]:
''' 
    Reproduction of subroutine update_strength_realtime
    from the COSMO code.
        Inputs: 
        Outputs:
        Steps: 1) [x] Load data from stations and from the model 
               2) [x] Treat missing data from stations 
               3) [x] Find cell indices of stations from lat-lon info
               4) [x] Modify tune_vec according to stations info
               5) [x] Interpolate over all grid cells
        TODO:  [x] Treat missing data from stations
               [x] INTERPOLATION: Implement IDW as is done in COSMO for different pollen types
               [x] Put everything as functions
               [x] Create python script in src/
               [ ] x Do writing in GRIB file -> commit
               [x] Implement the other FORTRAN function and adapt the common functions
               [x] check with 1 hr later output
               [x] check if tune_vec is not also used as input in interpolation
                
               
               [ ] Adapt Input management to other cases than ALNU/ICON input
               [ ] test on ICON input with every pollen type
'''

file_data = '/scratch/gvanpary/pollen/data/atabs/alnu_pollen_measured_values_2022020805.atab'
file_data_mod = '/scratch/gvanpary/pollen/data/atabs/alnu_pollen_modelled_values_2022022207.atab'

data, data_mod, lat_stns, lon_stns, missing_value, istation_mod = read_atab(file_data, file_data_mod)

pollen_types = ['ALNU', 'BETU', 'POAC', 'CORY']
ipollen = 0
array = data[data['PARAMETER'] == pollen_types[ipollen]].iloc[:,2:].to_numpy()
array_mod = data_mod[data_mod['PARAMETER'] == pollen_types[ipollen]].iloc[:,2:].to_numpy()

'''
    READING DATA FROM GRIB FILE
'''

ds = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022022207_ALNUtune',
        encode_cf=("time", "geography", "vertical")
    )
ds2 = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022022208_ALNUtune',
        encode_cf=("time", "geography", "vertical")
    )
array = treat_missing(array, missing_value, verbose = True)
change_tune = get_change_tune(pollen_types[ipollen], array, array_mod, ds, lat_stns, lon_stns, istation_mod)
change_tune_vec = interpolate2(change_tune, ds, lat_stns, lon_stns)
tune_vec_OLD = interpolate(change_tune, ds, 'ALNUtune', lat_stns, lon_stns, method= 'multiply', ipollen = 0)

Nstations = len(lon_stns)
tune_OLD = np.zeros(Nstations)
tune_next = np.zeros(Nstations)
for istation in range(Nstations):
    tune_OLD[istation] = get_field_at(ds, 'ALNUtune', lat_stns[istation], lon_stns[istation])
    tune_next[istation] = get_field_at(ds2,'ALNUtune', lat_stns[istation], lon_stns[istation])
change_tune_2 = tune_next/tune_OLD
change_tune_vec_2 = interpolate2(change_tune_2, ds, lat_stns, lon_stns)
tune_vec_OLD_2 = interpolate(change_tune_2, ds, 'ALNUtune', lat_stns, lon_stns, method= 'multiply', ipollen = 0)
#plot_tunevec(ds, 'ALNUtune', change_tune_vec, lat_stns, lon_stns)

In [ ]:
plt.figure()
plt.plot(change_tune, 'o')
plt.plot(change_tune_2, 'o')
plt.show()

In [ ]:
# T - T+1

plot_tunevec(ds, 'ALNUtune', ds.ALNUtune - ds2.ALNUtune, lat_stns, lon_stns, mode =2)

# gauss interp - T+1 

plot_tunevec(ds, 'ALNUtune', change_tune_vec*ds.ALNUtune - ds2.ALNUtune, lat_stns, lon_stns, mode =2)

# cosmo interp - T+1 
plot_tunevec(ds, 'ALNUtune', tune_vec_OLD - ds2.ALNUtune, lat_stns, lon_stns, mode =2)

# gauss interp (change_tune from T+1/T) - T+1
plot_tunevec(ds, 'ALNUtune', change_tune_vec_2*ds.ALNUtune - ds2.ALNUtune, lat_stns, lon_stns, mode =2)

# cosmo interp (change_tune from T+1/T) - T+1
plot_tunevec(ds, 'ALNUtune', tune_vec_OLD_2 - ds2.ALNUtune, lat_stns, lon_stns, mode =2)

In [ ]:
get_field_at(ds, 'vec', lat_stns[0], lon_stns[0])*get_field_at(ds, 'ALNUtune', lat_stns[0], lon_stns[0])

In [ ]:
print(get_field_at(ds, 'ALNUtune', lat_stns[0], lon_stns[0])*change_tune[0])

In [ ]:
Err = test_change_tune('ALNU', change_tune, ds, ds2, lat_stns, lon_stns, eps = 1e-2)

In [ ]:
change_tune

In [ ]:
# test other inputs
# BETU test case
file_data = '/scratch/gvanpary/pollen/data/atabs/betu_pollen_measured_values_2022032309.atab'
file_data_mod = '/scratch/gvanpary/pollen/data/atabs/betu_pollen_modelled_values_2022032309.atab'
data, data_mod, lat_stns, lon_stns, missing_value, istation_mod = read_atab(file_data, file_data_mod)
pollen_types = ['ALNU', 'BETU', 'POAC', 'CORY']
ipollen = 1
array = data[data['PARAMETER'] == pollen_types[ipollen]].iloc[:,2:].to_numpy()
array_mod = data_mod[data_mod['PARAMETER'] == pollen_types[ipollen]].iloc[:,2:].to_numpy()
ds = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022032309_BETUtune',
        encode_cf=("time", "geography", "vertical")
    )
ds2 = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022032310_BETUtune',
        encode_cf=("time", "geography", "vertical")
    )

array = treat_missing(array, missing_value, verbose = True)
change_tune = get_change_tune(pollen_types[ipollen], array, array_mod, ds, lat_stns, lon_stns, istation_mod)
tune_vec = interpolate2(change_tune, ds, lat_stns, lon_stns)
plot_tunevec(ds, 'BETUtune', change_tune, tune_vec, lat_stns, lon_stns, delta_tune= True)
Err = test_change_tune('BETU', change_tune, ds, ds2, lat_stns, lon_stns, eps = 1e-2)
print(Err)

In [ ]:
# test other inputs
# CORY test case
file_data = '/scratch/gvanpary/pollen/data/atabs/cory_pollen_measured_values_2022021110.atab'
file_data_mod = '/scratch/gvanpary/pollen/data/atabs/cory_pollen_modelled_values_2022021110.atab'
data, data_mod, lat_stns, lon_stns, missing_value, istation_mod = read_atab(file_data, file_data_mod)
pollen_types = ['ALNU', 'BETU', 'POAC', 'CORY']
ipollen = 3
array = data[data['PARAMETER'] == pollen_types[ipollen]].iloc[:,2:].to_numpy()
array_mod = data_mod[data_mod['PARAMETER'] == pollen_types[ipollen]].iloc[:,2:].to_numpy()
ds = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022021110_CORYtune',
        encode_cf=("time", "geography", "vertical")
    )
ds2 = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022021111_CORYtune',
        encode_cf=("time", "geography", "vertical")
    )

array = treat_missing(array, missing_value, verbose = True)
change_tune = get_change_tune(pollen_types[ipollen], array, array_mod, ds, lat_stns, lon_stns, istation_mod)
#tune_vec = interpolate2(change_tune, ds, lat_stns, lon_stns)
#plot_tunevec(ds, 'CORYtune', change_tune, tune_vec, lat_stns, lon_stns, delta_tune= True)
#Err = test_change_tune('CORY', change_tune, ds, ds2, lat_stns, lon_stns, eps = 1e-2)
#print(Err)

tune_vec = interpolate(change_tune, ds, 'CORYtune', lat_stns, lon_stns, method= 'mult', ipollen=ipollen)
Err = tune_vec - ds2.CORYtune
plot_tunevec(ds, 'CORYtune', tune_vec, tune_vec, lat_stns, lon_stns)
plot_tunevec(ds2, 'CORYtune', ds2.CORYtune, tune_vec, lat_stns, lon_stns)
plot_tunevec(ds2, 'CORYtune', Err, tune_vec, lat_stns, lon_stns)

In [ ]:
plt.figure()
plt.plot(Err.values.flatten()[0:-1:500], '.')
#plt.plot((ds.CORYtune - ds2.CORYtune).values.flatten()[0:-1:500], '.')
plt.show()

In [ ]:
# check uncertainty linked to the station position
# CORY test case
file_data = '/scratch/gvanpary/pollen/data/atabs/cory_pollen_measured_values_2022021110.atab'
file_data_mod = '/scratch/gvanpary/pollen/data/atabs/cory_pollen_modelled_values_2022021110.atab'
data, data_mod, lat_stns, lon_stns, missing_value, istation_mod = read_atab(file_data, file_data_mod)
pollen_types = ['ALNU', 'BETU', 'POAC', 'CORY']
ipollen = 3
array = data[data['PARAMETER'] == pollen_types[ipollen]].iloc[:,2:].to_numpy()
array_mod = data_mod[data_mod['PARAMETER'] == pollen_types[ipollen]].iloc[:,2:].to_numpy()
ds = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022021110_CORYtune',
        encode_cf=("time", "geography", "vertical")
    )
ds2 = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022021111_CORYtune',
        encode_cf=("time", "geography", "vertical")
    )

array = treat_missing(array, missing_value, verbose = True)

# now instead of getting one change tune, we get 5 (one for each neighbour cell of a 2D grid
# here we consider the COSMO grid for this test)

dlat = 1
dlon = 1
# first get the one as usual
def get_field_at(ds, field, lat, lon, eps = 1e-2):
    dist = (ds.latitude - lat)**2 + (ds.longitude - lon)**2
    return ds[field].where(dist == dist.min(), drop = True)

change_tune = get_change_tune(pollen_types[ipollen], array, array_mod, ds, lat_stns, lon_stns, istation_mod)

def get_field_at(ds, field, lat, lon, eps = 1e-2):
    dist = (ds.latitude - lat + dlat)**2 + (ds.longitude - lon)**2
    return ds[field].where(dist == dist.min(), drop = True)

change_tune_2 = get_change_tune(pollen_types[ipollen], array, array_mod, ds, lat_stns, lon_stns, istation_mod)

def get_field_at(ds, field, lat, lon, eps = 1e-2):
    dist = (ds.latitude - lat - dlat)**2 + (ds.longitude - lon)**2
    return ds[field].where(dist == dist.min(), drop = True)

change_tune_3 = get_change_tune(pollen_types[ipollen], array, array_mod, ds, lat_stns, lon_stns, istation_mod)

def get_field_at(ds, field, lat, lon, eps = 1e-2):
    dist = (ds.latitude - lat)**2 + (ds.longitude - lon + dlon)**2
    return ds[field].where(dist == dist.min(), drop = True)

change_tune_4 = get_change_tune(pollen_types[ipollen], array, array_mod, ds, lat_stns, lon_stns, istation_mod)

def get_field_at(ds, field, lat, lon, eps = 1e-2):
    dist = (ds.latitude - lat)**2 + (ds.longitude - lon - dlon)**2
    return ds[field].where(dist == dist.min(), drop = True)

change_tune_5 = get_change_tune(pollen_types[ipollen], array, array_mod, ds, lat_stns, lon_stns, istation_mod)

print(change_tune)
print(change_tune_2)
print(change_tune_3)
print(change_tune_4)
print(change_tune_5)

In [ ]:
ds2['diffr'] = ds2.ALNUtune.copy(deep=True)
ds2.diffr.values = ds2.ALNUtune - ds.tune_vec

ds2['diff_t'] = ds2.ALNUtune.copy(deep=True)
ds2.diff_t.values = ds2.ALNUtune - ds.ALNUtune

ds2['diff_t2'] = ds2.ALNUtune.copy(deep=True)
ds2.diff_t2.values = ds.tune_vec - ds.ALNUtune 

In [ ]:
ds = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/laf2022020805_filtered',
        encode_cf=("time", "geography", "vertical")
    )
ds

In [ ]:
ds = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/laf2022020805_filtered',
        encode_cf=("time", "geography", "vertical")
    )

DATE = pd.Timestamp(ds.time.values).day_of_year + 1 + 31
jul_days_excl = {"ALNUctsum": 14, "BETUctsum": 40, "CORYctsum": 3, "POACctsum": 46}
jul_days_excl = [14,40,3,46]
eps = 1e-2
Nstations = array.shape[1]
change_tthrs = np.zeros(Nstations)
change_tthre = np.zeros(Nstations)
verbose = True
for istation in range(Nstations):
    tthrs_stns = get_field_at(ds, pollen_types[ipollen] + 'tthrs', lon_stns[istation], lat_stns[istation], eps)
    tthre_stns = get_field_at(ds, pollen_types[ipollen] + 'tthre', lon_stns[istation], lat_stns[istation], eps)
    saisn_stns = get_field_at(ds, pollen_types[ipollen] + 'saisn', lon_stns[istation], lat_stns[istation], eps)
    ctsum_stns = get_field_at(ds, pollen_types[ipollen] + 'ctsum', lon_stns[istation], lat_stns[istation], eps)
    T_2M_stns = get_field_at(ds, 'T_2M', lon_stns[istation], lat_stns[istation], eps) - 273.15
    sum_obs_24 = np.sum(array[96:,istation])
    sum_obs = np.sum(array[:,istation])
    print(f'Current station n°{istation}, last 24H obs {sum_obs_24}, and last 120H {sum_obs}')
    print(f'Cumulative temperature sum {ctsum_stns} and threshold: {tthrs_stns}')
    # ADJUSTMENT OF SEASON START AND END AT THE BEGINNING OF THE SEASON
    if ((sum_obs_24 >= 240) and (sum_obs >= 720) and ctsum_stns < tthrs_stns):
        change_tthrs[istation] = (ctsum_stns - tthrs_stns)
        change_tthre[istation] = (ctsum_stns - tthrs_stns)
        if verbose:
            print(f'Big data and below threshold')
    elif ((sum_obs_24 < 240) and (sum_obs_24 >= 0) \
        and (sum_obs < 720) and (sum_obs >= 0)\
        and (ctsum_stns > tthrs_stns) and (ctsum_stns > tthre_stns)\
        and (saisn_stns < 10) and (saisn_stns > 0)):
        if verbose:
            print(f'Low data and in first 10 days of season')
        change_tthrs[istation] = T_2M_stns*(DATE - jul_days_excl[0])
        change_tthre[istation] = T_2M_stns*(DATE - jul_days_excl[0])
        
    # ADJUSTMENT OF SEASON END AT THE END OF THE SEASON
    if ((np.sum(array[96:,istation]) < 240) and (np.sum(array[96:,istation]) >= 0)\
        and (np.sum(array[:,istation]) < 720) and (np.sum(array[:,istation]) >= 0)\
        and (ctsum_stns + T_2M_stns*5*DATE > tthre_stns)\
        and (ctsum_stns < tthre_stns)):
        change_tthre[istation] += (ctsum_stns - tthre_stns)
        if verbose:
            print(f'Big data and below threshold')
    elif ((np.sum(array[96:,istation]) > 240) and (np.sum(array[:,istation]) > 720)\
        and (ctsum_stns + T_2M_stns*5*DATE > tthre_stns)\
        and (ctsum_stns < tthre_stns)):
        change_tthre[istation] += T_2M_stns*(DATE - jul_days_excl[0])
    # FAILSAFE 
    if (change_tthrs[istation] > 0):
        change_tthrs[istation] = min(1000, change_tthrs[istation])
    elif (change_tthrs[istation] <= 0):
        change_tthrs[istation] = max(-1000, change_tthrs[istation])

tthrs_vec = interpolate(change_tthrs, ds, 'ALNUtthrs', lat_stns, lon_stns, 'sum')
tthre_vec = interpolate(change_tthre, ds, 'ALNUtthre', lat_stns, lon_stns, 'sum')
plot_tunevec(ds, 'ALNUtthrs', change_tthrs, tthrs_vec, lon_stns, lat_stns)
plot_tunevec(ds, 'ALNUtthre', change_tthre, tthre_vec, lon_stns, lat_stns)



In [ ]:
ds2 = cfgrib.open_dataset(
        '/scratch/gvanpary/pollen/data/laf2022020806_filtered',
        encode_cf=("time", "geography", "vertical")
    )

ds2['diffr'] = ds2.ALNUtthrs.copy(deep=True)
ds2.diffr.values = ds2.ALNUtthre -tthre_vec

lonmin = np.amin(ds.longitude)
lonmax = np.amax(ds.longitude)
latmin = np.amin(ds.latitude)
latmax = np.amax(ds.latitude)


plot2= ds2.psy.plot.mapplot(
name="diffr",
title="tthrs(t+1) - Python",
titlesize=15,
map_extent = [lonmin, lonmax, latmin, latmax],
bounds = {'method': 'minmax', 'N':100},
lakes=True,
borders=True,
rivers=True,
cticksize=8,
clabel="Tuning Factor []",
grid_labelsize=8,
projection='robin',
cmap='RdBu_r', xgrid = False, ygrid = False)

In [ ]:
change_tune_vec = interpolate2(change_tune, ds, lat_stns, lon_stns, 200)
print(f'Max change tune at stations: {np.amax(change_tune)}')
print(f'Min change tune at stations: {np.amin(change_tune)}')
print(f'Max change tune extrapolation: {np.amax(change_tune_vec)}')
print(f'Min change tune extrapolation: {np.amin(change_tune_vec)}')

In [ ]:
plot_tunevec(ds, 'ALNUtune', change_tune, change_tune_vec, lat_stns, lon_stns, filename = 'delta_tune_RBF_2', delta_tune=True)

In [ ]:
plot_tunevec(ds2, 'ALNUtune', change_tune, ds2.ALNUtune, lat_stns, lon_stns)
plot_tunevec(ds, 'ALNUtune', change_tune, change_tune_vec*ds.ALNUtune, lat_stns, lon_stns)